In [158]:
import pandas as pd
import numpy as np

In [159]:
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 700)
pd.set_option('display.width', 450)

In [160]:
def get_start_stop_indices_state(df):
    state_idxs = df['STATE'].str.findall(r'\w\w')

    # build list of start and stop indices for states
    state_start_end_indices = []
    for s in state_idxs.items():
        if s[1]:
            state_start_end_indices.append(s[0])

    # create list of (start,end) tuples
    l_of_start_end_tuples = []
    for i in range(len(state_start_end_indices)-1):
        l_of_start_end_tuples.append((state_start_end_indices[i], state_start_end_indices[i+1]))
        
    # need to account for last index of state and add last tuple
    l_of_start_end_tuples.append((l_of_start_end_tuples[-1][1],len(state_idxs)-1))
    
    return l_of_start_end_tuples  

In [161]:
def get_start_stop_indices_counties_in_state(state_df):
    cty_idxs = state_df['County_name'].str.findall(r'(\w+\s?\w.*)')
    #print(cty_idxs)
    
    n_total_rows_for_all_counties = len(list(cty_idxs))
    #print("n_total_rows_for_all_counties:", n_total_rows_for_all_counties)
    
    cty_start_end_indices = []
    for c in cty_idxs.items():
        if c[1]:
            cty_start_end_indices.append(c[0])

    n_counties = len(cty_start_end_indices)

    # create list of (start,end) tuples
    l_of_start_end_tuples = []
    for i in range(n_counties-1):    
        l_of_start_end_tuples.append((cty_start_end_indices[i], cty_start_end_indices[i+1]))

    l_of_start_end_tuples.append( (l_of_start_end_tuples[-1][1], n_total_rows_for_all_counties) )

    return l_of_start_end_tuples 

In [162]:
df = pd.read_csv("data/homes_example.csv")
df = df.replace(np.nan, '', regex=True)
#df

In [163]:
def add_new_columns(state_df):

    #state_df['r1'] = ''
    #state_df['r3'] = ''
    state_df['rel'] = ''
    return state_df

In [164]:
def process_county(county_df):
    # check if rating of 1 or 2 does not exist first
    ratings = county_df['Comp5star']
    r = ratings[ratings != '']
    #print("ratings:\n",r)
    
    if (1 not in r.values) and (2 not in r.values):
        print("NO 1 and NO 2")
        county_df.loc[0:0,'rel'] = 'N/A'
       
    return county_df

In [165]:
state_idxs = get_start_stop_indices_state(df)
print("state start-end indicies:",state_idxs)
print()

for idx in state_idxs:
    start = idx[0]
    end   = idx[1]
    print("////////////////////////////////////////////////////////////////////////////////////////////\n")
    state_df = (df[start:end]).reset_index(drop=True)
    state_df = add_new_columns(state_df)
    print(state_df)
    print("============================================================================================\n")
    
    counties_idxs = get_start_stop_indices_counties_in_state(state_df)
    #print("\tcounty start-end indicies:", counties_idxs)
    for idx in counties_idxs:
        start  = idx[0]
        end    = idx[1]
        county_df = (state_df[start:end]).reset_index(drop=True)
        county = process_county(county_df)
        print(county)
        print("-----------------------------------------------------------------------------------------\n")

    

    

state start-end indicies: [(0, 4), (4, 10), (10, 18), (18, 21)]

////////////////////////////////////////////////////////////////////////////////////////////

  STATE           County_name Comp5star  PROVNUM PROVNAME  Sum of MDScensus_avg  Sum of Beds_Avail  Count of FACID rel
0    AK             Anchorage         4    25018        A                    10                 16               1    
1                                     5    25025        B                    11                 17               1    
2                                          25036        C                    12                 18               1    
3        Fairbanks North Star         3    25020        D                    13                 19               1    

NO 1 and NO 2
  STATE County_name Comp5star  PROVNUM PROVNAME  Sum of MDScensus_avg  Sum of Beds_Avail  Count of FACID  rel
0    AK   Anchorage         4    25018        A                    10                 16               1  N/A
1          

IndexError: list index out of range